In [6]:
import argparse
import os
import os.path as osp
import time
import cv2
import torch
import pdb

import sys
import numpy as np
import pyzed.sl as sl
import time

from loguru import logger

from yolox.data.data_augment import preproc
from yolox.exp import get_exp
from yolox.utils import fuse_model, get_model_info, postprocess
from yolox.utils.visualize import plot_tracking
from trackers.ocsort_tracker.ocsort import OCSort
from trackers.tracking_utils.timer import Timer

from threading import Lock, Thread

In [7]:
IMAGE_EXT = [".jpg", ".jpeg", ".webp", ".bmp", ".png"]

from utils.args import make_parser

In [8]:
# zedcamera thread control signal
exit_signal = False
new_data = False

In [9]:
# Tread lock intialize
lock = Lock()

In [11]:
# image processing fromo zed api to numpy array type
def load_image_into_numpy_array(image):
    """_summary_

    Args:
        image (zedAPI image): image made from zedAPI

    Returns:
        numpy array: numpy image
    """
    ar = image.get_data()
    ar = ar[:, :, 0:3]
    (im_height, im_width, channels) = image.get_data().shape
    return np.array(ar).reshape((im_height, im_width, 3)).astype(np.uint8)


def load_depth_into_numpy_array(depth):
    """_summary_

    Args:
        depth image (zedAPI image): depth image made from zedAPI

    Returns:
        numpy array: numpy depth image
    """
    ar = depth.get_data()
    ar = ar[:, :, 0:4]
    (im_height, im_width, channels) = depth.get_data().shape
    return np.array(ar).reshape((im_height, im_width, channels)).astype(np.float32)

In [12]:
# zed image output size
width = 704
height = 416
confidence = 0.35

# make global dummy image variables to contain zed images
image_np_global = np.zeros([width, height, 3], dtype=np.uint8)
depth_np_global = np.zeros([width, height, 4], dtype=np.float)

/tmp/ipykernel_10134/2656277456.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  depth_np_global = np.zeros([width, height, 4], dtype=np.float)


In [13]:
# ZED image capture thread function (zed cuda 분리를 위한 thread 생성, 안하면 pythorch와 cuda context 차이로 cuda error 생김)
def capture_thread_func(svo_filepath=None):
    global image_np_global, depth_np_global, exit_signal, new_data, point_cloud
    get_cloud = True
    zed = sl.Camera()

    # Create a InitParameters object and set configuration parameters
    input_type = sl.InputType()
    if svo_filepath is not None:
        input_type.set_from_svo_file(svo_filepath)

    init = sl.InitParameters(input_t=input_type)
    init.camera_resolution = sl.RESOLUTION.HD720
    init.camera_fps = 30
    init.depth_mode = sl.DEPTH_MODE.PERFORMANCE
    init.coordinate_units = sl.UNIT.MILLIMETER
    init.svo_real_time_mode = False


    # Open the camera
    err = zed.open(init)
    while err != sl.ERROR_CODE.SUCCESS:
        err = zed.open(init)
        print(err)
        exit(1)
    """
    image_mat = sl.Mat()
    depth_mat = sl.Mat()
    """
    runtime = sl.RuntimeParameters()
    runtime.sensing_mode = sl.SENSING_MODE.STANDARD
    
    image_size = zed.get_camera_information().camera_resolution
    image_size.width = image_size.width /2
    image_size.height = image_size.height /2
    
    image_zed = sl.Mat(image_size.width, image_size.height, sl.MAT_TYPE.U8_C4)
    depth_image_zed = sl.Mat(image_size.width, image_size.height, sl.MAT_TYPE.U8_C4)
    point_cloud = sl.Mat()
    

    while not exit_signal:
        if zed.grab(runtime) == sl.ERROR_CODE.SUCCESS:
            zed.retrieve_image(image_zed, sl.VIEW.LEFT, resolution=image_size)
            zed.retrieve_measure(depth_image_zed, sl.MEASURE.XYZRGBA, resolution=image_size)
            lock.acquire()
            image_np_global = load_image_into_numpy_array(image_zed)
            depth_np_global = load_depth_into_numpy_array(depth_image_zed)
            if get_cloud:
                zed.retrieve_measure(point_cloud, sl.MEASURE.XYZRGBA, sl.MEM.CPU, image_size)
            new_data = True
            lock.release()

        time.sleep(0.01)

    zed.close()

In [ ]:
# image 파일이 데이터 셋일경우 리스트로 만들어주는 함수
def get_image_list(path):
    image_names = []
    for maindir, subdir, file_name_list in os.walk(path):
        for filename in file_name_list:
            apath = osp.join(maindir, filename)
            ext = osp.splitext(apath)[1]
            if ext in IMAGE_EXT:
                image_names.append(apath)
    return image_names

In [14]:
class Predictor(object):
    def __init__(
        self,
        args,
        exp,
        trt_file=None,
        decoder=None,
        device=torch.device("cpu"),
        fp16=False
    ):
        
        self.decoder = decoder
        self.num_classes = exp.num_classes
        self.confthre = exp.test_conf
        self.nmsthre = exp.nmsthre
        self.test_size = exp.test_size
        #self.test_size = (1920, 1080) 테스트 파일 이미지 크기는 yolox/data/data_augment.py의 preproc 함수를 통해서 만들어짐
        self.device = device
        self.fp16 = fp16
        
        if trt_file is not None:
            from torch2trt import TRTModule
        
            model_trt = TRTModule()
            model_trt.load_state_dict(torch.load(trt_file))
            self.model = model_trt
        
        else:
            self.model = exp.get_model().to(args.device)
            
        self.rgb_means = (0.485, 0.456, 0.406)
        self.std = (0.229, 0.224, 0.225)

    def inference(self, img, timer):
        img_info = {"id": 0}
        if isinstance(img, str):
            img_info["file_name"] = osp.basename(img)
            img = cv2.imread(img)
        else:
            img_info["file_name"] = None

        height, width = img.shape[:2]
        img_info["height"] = height
        img_info["width"] = width
        img_info["raw_img"] = img

        img, ratio = preproc(img, self.test_size, self.rgb_means, self.std)
        img_info["ratio"] = ratio
        
        img = torch.from_numpy(img).unsqueeze(0).float().to(self.device)
        if self.fp16:
            img = img.half()  # to FP16


        with torch.no_grad():
            timer.tic()
            outputs = self.model(img)
            if self.decoder is not None:
                #pdb.set_trace()
                outputs = self.decoder(outputs, dtype=outputs.type())
                
            outputs = postprocess(
                outputs, self.num_classes, self.confthre, self.nmsthre
            )
        return outputs, img_info

In [ ]:
def depthflow_demo(predictor, vis_folder, current_time, args):
    global image_np_global, depth_np_global, new_data, exit_signal, point_cloud
    # 타임스텝 및 세이브 폴더
    timestamp = time.strftime("%Y_%m_%d_%H_%M_%S", current_time)
    save_folder = osp.join(vis_folder, timestamp)
    os.makedirs(save_folder, exist_ok=True)
    
    tracker = OCSort(det_thresh=args.track_thresh, iou_threshold=args.iou_thresh, use_byte=args.use_byte)
    timer = Timer()
    frame_id = 0
    results = []
    
    # Start the capture thread with the ZED input
    capture_thread = Thread(target=capture_thread_func)
    capture_thread.start()

    key = ' '
    prevTime = 0
    while key != 113 :
        # frame 분석 로거
        """
        if frame_id % 20 == 0:
            logger.info('Processing frame {} ({:.2f} fps)'.format(frame_id, 1. / max(1e-5, timer.average_time)))
        """
        
        if new_data:
            lock.acquire()
            image_ocv = np.copy(image_np_global)
            depth_image_zed = np.copy(depth_np_global)
            new_data = False
            lock.release()
            
            outputs, img_info = predictor.inference(image_ocv, timer)
            if outputs[0] is not None:
                online_targets = tracker.update(outputs[0], [img_info['height'], img_info['width']], exp.test_size)
                online_tlwhs = []
                online_ids = []
                for t in online_targets:
                    tlwh = [t[0], t[1], t[2] - t[0], t[3] - t[1]]
                    tid = t[4]
                    vertical = tlwh[2] / tlwh[3] > args.aspect_ratio_thresh
                    if tlwh[2] * tlwh[3] > args.min_box_area and not vertical:
                        online_tlwhs.append(tlwh)
                        online_ids.append(tid)
                        results.append(
                            f"{frame_id},{tid},{tlwh[0]:.2f},{tlwh[1]:.2f},{tlwh[2]:.2f},{tlwh[3]:.2f},1.0,-1,-1,-1\n"
                        )
                # timer.toc()
                online_im = plot_tracking(
                    img_info['raw_img'], point_cloud, online_tlwhs, online_ids, frame_id=frame_id + 1, fps=fps
                )
            else:
                timer.toc()
                online_im = img_info['raw_img']
        else:
            online_im = image_np_global
            #time.sleep(0.01)
        curTime = time.time()
        fps = int(1./(curTime - prevTime))
        prevTime = curTime
        cv2.imshow("image", online_im)
        key = cv2.waitKey(10)
    cv2.destroyAllWindows()
    capture_thread.join()
    print("\nFINISH")

In [ ]:
def main(exp, args):
    if not args.expn:
        args.expn = exp.exp_name

    output_dir = osp.join(exp.output_dir, args.expn)
    os.makedirs(output_dir, exist_ok=True)

    if args.save_result:
        vis_folder = osp.join(output_dir, "track_vis")
        os.makedirs(vis_folder, exist_ok=True)

    if args.trt:
        args.device = "gpu"
    args.device = torch.device("cuda" if args.device == "gpu" else "cpu")
    
    if args.trt:
        assert not args.fuse, "TensorRT model is not support model fusing!"
        trt_file = osp.join(output_dir, "model_trt.pth")
        assert osp.exists(
            trt_file
        ), "TensorRT model is not found!\n Run python3 tools/trt.py first!"
        model.head.decode_in_inference = False
        decoder = model.head.decode_outputs
        logger.info("Using TensorRT to inference")
    else:
        trt_file = None
        decoder = None

    logger.info("Args: {}".format(args))

    if args.conf is not None:
        exp.test_conf = args.conf
    if args.nms is not None:
        exp.nmsthre = args.nms
    if args.tsize is not None:
        exp.test_size = (args.tsize, args.tsize)
        
    predictor = Predictor(args, exp, trt_file, decoder, args.device, args.fp16)
    logger.info("Model Summary: {}".format(get_model_info(predictor.model, exp.test_size)))
    model = predictor.model
    model.eval()
    
    """
    # 여기에 model 초기화를 하니 trt 모델이 돌아갔습니다... 수정필요
    if args.trt:
        x = torch.ones((1, 3, 800, 1440), device="cuda")
        model(x)
    # print("=======================")
    # print(model.head.hw)
    """

    if not args.trt:
        if args.ckpt is None:
            ckpt_file = osp.join(output_dir, "best_ckpt.pth.tar")
        else:
            ckpt_file = args.ckpt
        logger.info("loading checkpoint")
        ckpt = torch.load(ckpt_file, map_location="cpu")
        # load the model state dict
        model.load_state_dict(ckpt["model"])
        logger.info("loaded checkpoint done.")

        if args.fuse:
            logger.info("\tFusing model...")
            model = fuse_model(model)

        if args.fp16:
            model = model.half()  # to FP16

    current_time = time.localtime()
    if args.demo_type == "image":
        image_demo(predictor, vis_folder, current_time, args)
    elif args.demo_type == "video" or args.demo_type == "webcam":
        imageflow_demo(predictor, vis_folder, current_time, args)
    elif args.demo_type == "depthcam":
        depthflow_demo(predictor, vis_folder, current_time, args)